In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import src.support_functions as sf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           confusion_matrix, classification_report, roc_auc_score, roc_curve)

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer


import joblib

In [ ]:
data = data = pd.read_csv('../data/CHURN_FEATURES.csv')

data = data.drop(columns=[
    'ID_VENDEDOR',
    'ID_CAMPANA',
    'ID_VENDEDOR.1',
    'ID_CAMPANA.1',
    'ID_VENDEDOR.2',
    'ID_CAMPANA.2',
    'ID_UBICACION',
    'ID_VENDEDOR.3',
    'ID_CAMPANA.3',
    'ID_VENDEDOR.4',
    'ID_CAMPANA.4',
    'ID_VENDEDOR.5',
    'ID_CAMPANA.5',
    'NUMERO_CAMPANA',
    'NUMERO_CAMPANA.1',
    'PROVINCIA',
    'DISTRITO',
    'ANIO'
])

data.drop(columns=['EDAD_VENDEDORA'], inplace=True, axis=1)
data['ANTIGUEDAD_MESES'] = data['ANTIGUEDAD_MESES'].clip(lower=0)
data.dropna(inplace=True)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC


num_features = [
    'ANTIGUEDAD_MESES',
    'MONTO_ACTUAL',
    'MONTO_ANTERIOR',
    'NUM_CAMPANAS_HISTORICAS',
    'CAMPANAS_CONSECUTIVAS_PREVIAS'
]
bin_features = [
    'SEXO',           # map {'F': 0, 'M': 1}
    'TIPO_VENDEDOR',  # map {'Asesora': 0, 'Líder': 1}
    'ES_NUEVA',
    'ES_INICIO_ANIO',
    'ES_NAVIDAD'
]

cat_features = ['DEPARTAMENTO']

def map_binarias(X):
    X = X.copy()
    X['SEXO'] = X['SEXO'].map({'F': 0, 'M': 1}).fillna(0)
    X['TIPO_VENDEDOR'] = X['TIPO_VENDEDOR'].map({'Asesora': 0, 'Líder': 1})
    return X

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features),
    ('bin', 'passthrough', bin_features) 
])
